In [ ]:
pwd

'/mnt/c/Users/tanub/Documents/NYU classes/BIOL GA 1009/Home_Work/Homework 5'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
import math

In [ ]:
!pip install plotly

     |████████████████████████████████| 13.2 MB 7.0 MB/s eta 0:00:01    |████████████████████            | 8.3 MB 7.0 MB/s eta 0:00:01     |████████████████████████▎       | 10.0 MB 7.0 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=88b902ec50dbd52222f9698ac0e17efd07a2dd3dfca27cbcddf49d776b6d00c4
  Stored in directory: /root/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [ ]:
cancer_data = pd.read_csv("data.csv", sep=",")

In [ ]:
cancer_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
# to perform z transformation, use StandardScaler().fit_transform()
size_nodspos_scale = StandardScaler().fit_transform(cancer_data[["radius_mean", "texture_mean", "smoothness_mean", "compactness_mean"]])

# Substitute the original data with the transformed data
cancer_data[["radius_mean", "texture_mean", "smoothness_mean", "compactness_mean"]] = size_nodspos_scale
# notice column "size" and "nodespos" have changed
cancer_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,1.097064,-2.073335,122.80,1001.0,1.568466,3.283515,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,1.829821,-0.353632,132.90,1326.0,-0.826962,-0.487072,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,1.579888,0.456187,130.00,1203.0,0.942210,1.052926,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,-0.768909,0.253732,77.58,386.1,3.283553,3.402909,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,1.750297,-1.151816,135.10,1297.0,0.280372,0.539340,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
specific_data = cancer_data[["diagnosis","radius_mean", "texture_mean", "smoothness_mean", "compactness_mean"]]

# set alivestatus as categorical
specific_data = specific_data.astype({'diagnosis': 'category'})

specific_data

,diagnosis,radius_mean,texture_mean,smoothness_mean,compactness_mean
0,M,1.097064,-2.073335,1.568466,3.283515
1,M,1.829821,-0.353632,-0.826962,-0.487072
2,M,1.579888,0.456187,0.942210,1.052926
3,M,-0.768909,0.253732,3.283553,3.402909
4,M,1.750297,-1.151816,0.280372,0.539340
...,...,...,...,...,...
564,M,2.110995,0.721473,1.041842,0.219060
565,M,1.704854,2.085134,0.102458,-0.017833
566,M,0.702284,2.045574,-0.840484,-0.038680
567,M,1.838341,2.336457,1.525767,3.272144


In [ ]:
# randomly choose 5000 rows as test set
data_test = specific_data.sample(250)

# set difference
whole_index = specific_data.index
test_index = data_test.index
train_index = whole_index.difference(test_index)

 
data_train = specific_data.loc[train_index,]

In [ ]:
x_test = np.array(data_test[["radius_mean", "texture_mean", "smoothness_mean", "compactness_mean"]])
y_test = np.array(data_test["diagnosis"])

x_train = np.array(data_train[["radius_mean", "texture_mean", "smoothness_mean", "compactness_mean"]])
y_train = np.array(data_train["diagnosis"])

k-value = 1

In [ ]:
data_knn1 = KNeighborsClassifier(n_neighbors=1).fit(x_train, y_train)

predicted1 = data_knn1.predict(x_test)

predicted_proba1 = data_knn1.predict_proba(x_test)

In [ ]:
fpr1, tpr1, threshold1 = roc_curve(y_test, predicted_proba1[:,1], pos_label="M")

roc_auc1 = auc(fpr1, tpr1)

print("AUC of the model with k-value of 1 is: ", roc_auc1)

AUC of the model with k-value of 1 is:  0.8586805555555557


k-value = 3

In [ ]:
data_knn2 = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)

predicted2 = data_knn2.predict(x_test)

predicted_proba2 = data_knn2.predict_proba(x_test)

In [ ]:
fpr2, tpr2, threshold2 = roc_curve(y_test, predicted_proba2[:,1], pos_label="M")

roc_auc2 = auc(fpr2, tpr2)

print("AUC of the model with k-value of 3 is: ", roc_auc2)

AUC of the model with k-value of 3 is:  0.9218402777777778


k-value = 10

In [ ]:
data_knn3 = KNeighborsClassifier(n_neighbors=10).fit(x_train, y_train)

predicted3 = data_knn3.predict(x_test)

predicted_proba3 = data_knn3.predict_proba(x_test)

In [ ]:
fpr3, tpr3, threshold3 = roc_curve(y_test, predicted_proba3[:,1], pos_label="M")

roc_auc3 = auc(fpr3, tpr3)

print("AUC of the model with k-value of 10 is: ", roc_auc3)

AUC of the model with k-value of 10 is:  0.970625


k-value = 50

In [ ]:
data_knn4 = KNeighborsClassifier(n_neighbors=50).fit(x_train, y_train)

predicted4 = data_knn4.predict(x_test)

predicted_proba4 = data_knn4.predict_proba(x_test)

In [ ]:
fpr4, tpr4, threshold4 = roc_curve(y_test, predicted_proba4[:,1], pos_label="M")

roc_auc4 = auc(fpr4, tpr4)

print("AUC of the model with k-value of 50 is: ", roc_auc4)

AUC of the model with k-value of 50 is:  0.9801041666666667


k-value of 50 gives best results!

In [ ]:
knn_df = data_test.copy()
#knn_df = data_train.copy()
knn_df["predict"] = predicted4

In [ ]:
knn_df

,diagnosis,radius_mean,texture_mean,smoothness_mean,compactness_mean,predict
466,B,-0.280405,0.337507,-0.683920,0.086400,B
138,M,0.233661,-0.400174,1.447485,0.495752,M
376,B,-1.010321,0.216499,-0.400681,1.168530,B
110,B,-1.235545,-0.535144,0.522334,-0.384734,B
329,M,0.605719,0.602792,1.433252,0.454059,M
...,...,...,...,...,...,...
343,M,1.577048,0.556251,0.114556,0.560187,M
191,B,-0.385490,0.493420,-0.631257,-0.726429,B
521,M,2.982918,0.537634,0.472519,2.013766,M
220,B,-0.135558,-1.426410,0.007097,-0.326553,B
